In [3]:
import mgclient

Connect to database

In [4]:
conn = mgclient.connect(host='127.0.0.1', port=7687)
cursor = conn.cursor()

Step 1<br>
This first step gets one of its inputs, properties, from Google Cloud Speech, containing speech recognition hypotheses and properties related to these. The mess_dict\['id'\] contains a unique integer ID denoting the human utterance.

In [ ]:
query = "CREATE (:ASRHypothesis {properties})<-[:ALTERNATIVE]-(:HumanUtterance {id: {mess_dict['id']}})"
cursor.execute(query)

Step 2<br>
rasa_nlu refers to a Natural Language Understanding module that extracts entities and intents from the given speech recognition hypothesis.

In [ ]:
query = "MATCH (u:HumanUtterance)-[:ALTERNATIVE]->(h:ASRHypothesis) " \
    + "CALL rasa_nlu.run(h.transcript) YIELD intents, entities " \
    + "UNWIND intents as intent " \
    + "MERGE (i:Intent {name:intent.name, prob:intent.prob})-[:INFERRED_FROM]->(h) " \
    + "WITH entities, h, i, size(entities) > 0 as len " \
    + "UNWIND CASE len WHEN TRUE THEN entities ELSE [{entity: 'dummy', value: 'dummy', prob: 0, start: 'No', end: 'No'}] END as entity " \
    + "MERGE (em:EntityMention {entity:entity.entity, value:entity.value, prob:entity.prob, start:entity.start, end:entity.end})-[:INFERRED_FROM]->(h);"
cursor.execute(query)

Step 3<br>
For now, we use the CONTAINS keyword in Cypher to detect whether the name of the entity mention refers to an actual person in the database. This is, however, not an optimal solution, and further processing of strings needs to be done to get an accurate match.


In [11]:
query = "MATCH (em:EntityMention {entity: 'person'}), (p:Person) " \
    + "WHERE toLower(p.name) CONTAINS toLower(em.value) " \
    + "MERGE (em)<-[:INFERRED_FROM]-(p);"
cursor.execute(query)

Step 4

In [12]:
query = "MATCH (em:EntityMention)<-[:INFERRED_FROM]-(p:Person) " \
    + "MATCH (ah:ASRHypothesis)<-[:INFERRED_FROM]-(em)<-[:INFERRED_FROM]-(p) " \
    + "MATCH (i:Intent {name: 'request_person_location'})-[:INFERRED_FROM]->(ah) " \
    + "MATCH (p)-[:LOCATED_IN]->(l:Location) " \
    + "MERGE (r:Response {name: 'inform_person_location'})-[:INFERRED_FROM]->(i) " \
    + "MERGE (r)<-[:ARG_OF]-(l) " \
    + "MERGE (r)<-[:ARG_OF]-(p);"
cursor.execute(query)

Step 5

In [13]:
query = "MATCH (r:Response {name: 'inform_person_location'})-[:ANSWERS_TO]->(:Intent {name: 'request_person_location'}) " \
    + "MATCH (r)<-[:ARG_OF]-(l:Location) " \
    + "MATCH (r)<-[:ARG_OF]-(p:Person) " \
    + "MERGE (ru:RobotUtterance {transcript: p.name + ' is at ' + l.name}) " \
    + "MERGE (ru)<-[:ARG_OF]-(p) " \
    + "MERGE (ru)<-[:ARG_OF]-(l) " \
    + "MERGE (ru)-[:SURFACE_REALISATION]->(r);"
cursor.execute(query)

In [14]:
conn.commit()